# Web application
Overview:
- install yfinance
- testing API
- deploying web app

## YFinance
Source of yfinance module: https://pypi.org/project/yfinance/ and https://github.com/ranaroussi/yfinance

Short usage instructions for our needs:
`sym = yf.Ticker('YAHOOSYMBOL')` and data retrieval `sym.history(period, interval, auto_adjust=True)`

- period : valid values 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
- interval : valid values 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo

In [6]:
!pip install yfinance

  Running setup.py bdist_wheel for yfinance ... done
  Stored in directory: /home/ec2-user/.cache/pip/wheels/f9/e3/5b/ec24dd2984b12d61e0abf26289746c2436a0e7844f26f2515c
  Running setup.py bdist_wheel for multitasking ... done
  Stored in directory: /home/ec2-user/.cache/pip/wheels/37/fa/73/d492849e319038eb4d986f5152e4b19ffb1bc0639da84d2677
Successfully built yfinance multitasking
fastai 1.0.59 requires nvidia-ml-py3, which is not installed.
You are using pip version 10.0.1, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


# Example
A short example of collecting data from the 2 sources we are interested in.

In [1]:
import yfinance as yf

In [8]:
sym_ixic = yf.Ticker('^IXIC')

In [9]:
sym_ixic.history(period='5d', interval='1d')

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2020-01-24,9446.21,9451.43,9273.23,9314.91,2611710000,0,0
2020-01-27,9092.46,9185.45,9088.04,9139.31,2583330000,0,0
2020-01-28,9201.82,9288.87,9182.33,9269.68,2157830000,0,0
2020-01-29,9318.26,9329.12,9249.04,9275.16,2223480000,0,0
2020-01-30,9211.15,9303.00,9185.18,9298.93,2333500000,0,0


In [10]:
sym_eurusd = yf.Ticker('EURUSD=X')

In [17]:
data = sym_eurusd.history(period='5d', interval='1d')
data

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2020-01-27,1.1026,1.1040,1.1010,1.1025,0,0,0
2020-01-28,1.1020,1.1027,1.0999,1.1020,0,0,0
2020-01-29,1.1023,1.1030,1.0993,1.1023,0,0,0
2020-01-30,1.1014,1.1037,1.1008,1.1017,0,0,0
2020-01-31,1.1038,1.1058,1.1020,1.1055,0,0,0


# Deployment For Web App
Here are the final steps for deploying the app.

Paste `model_data` from previous notebook, so correct model is loaded.

In [ ]:
# Below we specify which estimator we want to use - this should match estimator.model_data from previous notebook
model_data = 's3://sagemaker-us-east-1-236375122127/pytorch-training-2020-01-31-13-42-51-752/output/model.tar.gz'

In [4]:
import sagemaker

sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
prefix = 'sagemaker/stock_forecasting'

role = sagemaker.get_execution_role()

In [5]:
from sagemaker.predictor import RealTimePredictor
from sagemaker.pytorch import PyTorchModel

class JsonPredictor(RealTimePredictor):
    def __init__(self, endpoint_name, sagemaker_session):
        super().__init__(endpoint_name, sagemaker_session, content_type='application/json')

model = PyTorchModel(model_data=model_data,
                     role = role,
                     framework_version='1.2.0',
                     entry_point='predict.py',
                     source_dir='model',
                     predictor_cls=JsonPredictor)

In [6]:
predictor = model.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

---------------------!

## Lambda
We need to create a new lambda, to which we can connect our html page. Lambda will use runtime Python3.6 and requires User Role with Full SageMaker access.

Paste the endpoint into the lambda function, which you can find inside `webapp/lambda.py` file with correct endpoint name and save.

It is necessary to add a trigger. Select API Gateway HTTP and accept.
The endpoint should be listed on the bottom of the page. After expanding there is API endpoint url, which we will need in the `index.html` file.

In [8]:
predictor.endpoint

'pytorch-inference-2020-01-31-18-14-54-141'

In [7]:
!pygmentize webapp/lambda.py

import json
import boto3

def lambda_handler(event, context):
    runtime = boto3.Session().client('sagemaker-runtime')

    response = runtime.invoke_endpoint(EndpointName = '**ENDPOINT NAME**',
                                       ContentType = 'text/plain',
                                       Body = event['body'])

    result = json.loads(response['Body'].read())

    return {
        'statusCode' : 200,
        'headers' : { 'Content-Type' : 'application/json', 'Access-Control-Allow-Origin' : '*' },
        'body' : result
    }
